In [2]:
import pandas as pd
import numpy as np

In [56]:
data = {
    "사용자": ["A", "B", "C", "D"],
    "기생충": [5, 4, 1, 0],
    "부산행": [3, 4, 0, 4],
    "도둑들": [4, 0, 2, 5],
    "파묘": [3, 0, 3, 4]
}

df = pd.DataFrame(data)
df = df.set_index("사용자")
df

,기생충,부산행,도둑들,파묘
사용자,,,,
A,5,3,4,3
B,4,4,0,0
C,1,0,2,3
D,0,4,5,4


In [ ]:
# 경사하강법 -> 오차를 줄이기 위해서 학습 반복
# step -> 학습 횟수
# learning rate -> 보폭
# 정규화 정도를 선택 -> 정규화 파라미터 설정(L1 lasso, L2 lidge)
# 목표: 4 x 4 행렬(N x M 원본 행렬)을 4 x 2 행렬(N x K 사용자 요인 행렬)과 2 x 4 행렬(K x M 아이템 요인 행렬)의 곱으로 변환
# 1. 사용자 요인행렬을 랜덤행렬로 만들기 -> 학습하면서 오차를 줄여나가기
# 2. 아이템 요인행렬을 랜덤행렬로 만들기 -> 학습하면서 오차를 줄여나가기

In [97]:
R = df.values  # 본래 평점 데이터
K = 2  # 잠재요인 수
# steps = 5000
steps = 50000
alpha = 0.0002  # learning_rate
# alpha = 0.002  # learning_rate
beta = 0.02  # 정규화 옵션

M = len(R[0])
N = len(R)

# 사용자 잠재 요인 행렬 -> N x K 행렬 -> 랜덤으로 만들고 -> 학습
P = np.random.rand(N, K)
# P = np.random.rand(N, K) * 5

# 아이템 잠재 요인 행렬 -> K x M 행렬 -> 랜덤으로 만들고 -> 학습
Q = np.random.rand(K, M)
# Q = np.random.rand(K, M) * 5

print("초기 P:\n", P)
print("초기 Q:\n", Q)


초기 P:
 [[0.86858911 0.29756847]
 [0.0079593  0.35226544]
 [0.58067714 0.10324025]
 [0.72237594 0.01934291]]
초기 Q:
 [[0.29988332 0.45433295 0.6600541  0.69801503]
 [0.96418993 0.47973681 0.9377875  0.83644081]]


In [98]:
for step in range(steps):
    for i in range(N):
        for j in range(M):
            if R[i][j] > 0:  # 실제 평점이 있는 경우 예측 평점을 구하기
                eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                # print(eij)
                
                # 경사하강법으로 P와 Q를 업데이트
                for k in range(K):
                    P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                    Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
                
                # P와 Q의 값을 제한 (np.clip 사용)
                P[i] = np.clip(P[i], 0, 5)
                Q[:, j] = np.clip(Q[:, j], 0, 5)
                
    # error = np.dot(P, Q)
    e = 0
    for i in range(N):
        for j in range(M):
            if R[i][j] > 0:
                e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                for k in range(K):
                    e = e + (beta/2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
    if e < 0.001:
        break
    
    # # 예측 평점 제한 적용 (Clipping)
    # P = np.clip(P, 0, 5)  # 사용자 잠재 요인 행렬 범위 제한
    # Q = np.clip(Q, 0, 5)  # 아이템 잠재 요인 행렬 범위 제한

    print(f"Step {step}\tError: {e}")

Step 0	Error: 123.18006174242703
Step 1	Error: 122.80439516487773
Step 2	Error: 122.42813291499807
Step 3	Error: 122.05127331101805
Step 4	Error: 121.67381490336467
Step 5	Error: 121.29575647439789
Step 6	Error: 120.91709703813586
Step 7	Error: 120.53783583996704
Step 8	Error: 120.15797235634948
Step 9	Error: 119.77750629449551
Step 10	Error: 119.39643759204179
Step 11	Error: 119.01476641670287
Step 12	Error: 118.63249316590795
Step 13	Error: 118.24961846642039
Step 14	Error: 117.86614317393759
Step 15	Error: 117.48206837267271
Step 16	Error: 117.0973953749147
Step 17	Error: 116.71212572056807
Step 18	Error: 116.32626117667039
Step 19	Error: 115.93980373688686
Step 20	Error: 115.55275562098154
Step 21	Error: 115.16511927426384
Step 22	Error: 114.77689736701002
Step 23	Error: 114.38809279385872
Step 24	Error: 113.99870867317978
Step 25	Error: 113.60874834641533
Step 26	Error: 113.2182153773932
Step 27	Error: 112.82711355161071
Step 28	Error: 112.43544687548945
Step 29	Error: 112.0432195

In [99]:
# 사용자 잠재 요인 행렬
print("사용자 잠재 요인 행렬:\n", P)

# 아이템 잠재 요인 행렬
print("아이템 잠재 요인 행렬:\n", Q)

# 예측한 평점 데이터
nR = np.dot(P, Q)
print("예측한 평점 데이터:\n", nR)

사용자 잠재 요인 행렬:
 [[9.12326436e-01 1.79545642e+00]
 [1.86092968e+00 1.15138670e+00]
 [1.58513707e+00 8.87058726e-06]
 [1.39699722e+00 2.00595435e+00]]
아이템 잠재 요인 행렬:
 [[0.63915473 1.59933742 1.26246752 1.87260729]
 [2.43886197 0.87035385 1.59481788 0.69991644]]
예측한 평점 데이터:
 [[4.96198813 3.02180021 4.01520848 2.96509859]
 [3.99749523 3.97836831 4.18561537 4.29066495]
 [1.01316948 2.53517675 2.00119822 2.96834544]
 [5.78514316 3.98016003 4.96279548 4.02002761]]


In [100]:
# 예측한 평점 데이터 프레임
nR_df = pd.DataFrame(nR, index=df.index, columns=df.columns)
nR_df

# 예측 평점이 5점 이상이 되지 않도록 가중치 제약조건을 부여하는 방안 - reject?
# - 같은 5점이라도 사람마다 다르기 때문?

,기생충,부산행,도둑들,파묘
사용자,,,,
A,4.961988,3.021800,4.015208,2.965099
B,3.997495,3.978368,4.185615,4.290665
C,1.013169,2.535177,2.001198,2.968345
D,5.785143,3.980160,4.962795,4.020028


In [84]:
df

,기생충,부산행,도둑들,파묘
사용자,,,,
A,5,3,4,3
B,4,4,0,0
C,1,0,2,3
D,0,4,5,4
